# Energy-Backed Derivatives Pricing Framework
## Complete Demonstration and Analysis

**Author**: Energy Finance Research Group  
**Date**: November 2025  
**Purpose**: Demonstrates quantitative pricing of renewable energy-backed digital assets using binomial trees, Monte-Carlo simulation, and risk-neutral valuation.

---

### Project Overview

This notebook implements a complete derivative pricing framework for **energy-backed digital assets**:

1. **Theoretical Foundation**: Treat energy-backed tokens as European-style financial derivatives
2. **Pricing Methods**: Binomial trees and Monte-Carlo simulation
3. **Risk Analysis**: Greeks (Delta, Gamma, Vega, Theta, Rho)
4. **Empirical Integration**: Load Bitcoin CEIR data to calibrate models
5. **Stress Testing**: Evaluate robustness under different market conditions

### Connection to CEIR

The **Cumulative Energy Investment Ratio (CEIR)** framework establishes that:
- Energy costs create fundamental value anchors
- Renewable energy output has intrinsic economic value
- Digital assets can be priced using real-world energy as underlying

This notebook shows **how to price such assets rigorously**.

## Section 1: Setup and Data Loading

In [ ]:
# Import all modules
import sys
sys.path.insert(0, '../src')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Import our modules
from binomial import BinomialTree, price_energy_call, price_energy_claim
from monte_carlo import MonteCarloSimulator, price_energy_derivative_mc
from sensitivities import GreeksCalculator, compute_energy_derivatives_greeks
from plots import EnergyDerivativesPlotter
from data_loader import load_ceir_data, load_parameters, compute_energy_price, estimate_volatility, get_ceir_summary

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("✓ All modules imported successfully")
print(f"Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

### Load Empirical Data

In [ ]:
# Load CEIR data and parameters
params = load_parameters(data_dir='../empirical', T=1.0, r=0.05)

print("📊 Loaded Parameters:")
print(f"  S₀ (Initial Energy Price): ${params['S0']:.4f}")
print(f"  σ (Volatility): {params['sigma']:.2%}")
print(f"  T (Time to Maturity): {params['T']} year")
print(f"  r (Risk-Free Rate): {params['r']:.2%}")
print(f"  K (Strike Price): ${params['K']:.4f}")
print(f"\n📈 Data Summary:")

summary = get_ceir_summary(params['ceir_df'])
print(f"  Date Range: {summary['start_date'].date()} to {summary['end_date'].date()}")
print(f"  Trading Days: {summary['n_days']}")
print(f"  Current Price: ${summary['price_current']:.2f}")
print(f"  CEIR (Mean ± Std): {summary['ceir_mean']:.2f} ± {summary['ceir_std']:.2f}")

## Section 2: Binomial Tree Pricing

### 2.1 European Call Option on Energy

In [ ]:
# Create binomial tree for European call
S0 = params['S0']
K = S0  # At-the-money
T = params['T']
r = params['r']
sigma = params['sigma']

print("🌳 BINOMIAL TREE PRICING - European Call Option")
print(f"\nParameters:")
print(f"  S₀ = ${S0:.4f}")
print(f"  K = ${K:.4f}")
print(f"  T = {T} year")
print(f"  r = {r:.2%}")
print(f"  σ = {sigma:.2%}")

# Price with different step counts
print("\n📊 Convergence Table:")
tree_100 = BinomialTree(S0, K, T, r, sigma, N=100, payoff_type='call')
convergence_df = tree_100.sensitivity_analysis_convergence([10, 25, 50, 100, 200, 500])
print(convergence_df.to_string(index=False))

call_price = tree_100.price()
print(f"\n✓ European Call Price (N=100): ${call_price:.4f}")

### 2.2 Direct Redeemable Energy Claim

In [ ]:
# Price direct redeemable claim
tree_redeemable = BinomialTree(S0, K, T, r, sigma, N=100, payoff_type='redeemable')
redeemable_price = tree_redeemable.price()

print("🌳 BINOMIAL TREE PRICING - Direct Redeemable Claim")
print(f"\nPayoff: S_T (direct 1-unit energy claim)")
print(f"\n✓ Redeemable Claim Price (N=100): ${redeemable_price:.4f}")
print(f"\n📊 Comparison:")
print(f"  European Call:      ${call_price:.4f}")
print(f"  Redeemable Claim:   ${redeemable_price:.4f}")
print(f"  Difference:         ${redeemable_price - call_price:.4f}")

## Section 3: Monte-Carlo Simulation

In [ ]:
# Monte-Carlo pricing
print("🎲 MONTE-CARLO SIMULATION")
print(f"\nNumber of Simulations: 10,000 paths")

mc_sim = MonteCarloSimulator(S0, K, T, r, sigma, num_simulations=10000, payoff_type='call')
mc_price, mc_low, mc_high = mc_sim.confidence_interval()

print(f"\n✓ Monte-Carlo Price: ${mc_price:.4f}")
print(f"  95% Confidence Interval: [${mc_low:.4f}, ${mc_high:.4f}]")
print(f"  Interval Width: ${mc_high - mc_low:.4f}")

# Compare methods
print(f"\n📊 METHOD COMPARISON:")
print(f"  Binomial (N=100):   ${call_price:.4f}")
print(f"  Monte-Carlo (10k):  ${mc_price:.4f}")
print(f"  Difference:         ${abs(call_price - mc_price):.4f}")
print(f"  Rel. Difference:    {abs(call_price - mc_price) / call_price * 100:.2f}%")

### Distribution Analysis

In [ ]:
# Get distribution statistics
dist_stats = mc_sim.price_distribution()
print("📈 Terminal Payoff Distribution Statistics:")
print(dist_stats.to_string())

## Section 4: Greeks Calculation (Sensitivity Analysis)

In [ ]:
# Compute Greeks
print("🔍 GREEKS CALCULATION - Risk Sensitivities")
print(f"\nGreeks measure how option price changes with market parameters:")
print(f"\n  δ (Delta):     Price sensitivity to underlying (∂V/∂S)")
print(f"  Γ (Gamma):     Delta's sensitivity to underlying (∂²V/∂S²)")
print(f"  ν (Vega):      Price sensitivity to volatility (∂V/∂σ)")
print(f"  θ (Theta):     Time decay per day (∂V/∂T)")
print(f"  ρ (Rho):       Interest rate sensitivity (∂V/∂r)")

greeks_calc = GreeksCalculator(S0, K, T, r, sigma, pricing_method='binomial', N=50, payoff_type='call')
greeks_df = greeks_calc.to_dataframe()

print(f"\n" + "="*80)
print(greeks_df.to_string(index=False))
print("="*80)

### Interpret Greeks

In [ ]:
greeks = greeks_calc.compute_all_greeks()

print("\n💡 GREEKS INTERPRETATION FOR THIS ENERGY DERIVATIVE:\n")
print(f"1. DELTA = {greeks['Delta']:.4f}")
print(f"   → The option price increases by ${greeks['Delta']:.4f} for each $1 increase in energy price")
print(f"   → Represents hedge ratio (units of underlying per option)\n")

print(f"2. GAMMA = {greeks['Gamma']:.4f}")
print(f"   → Delta changes by {greeks['Gamma']:.4f} for each $1 move in energy price")
print(f"   → Measures convexity; positive = favorable gamma\n")

print(f"3. VEGA = {greeks['Vega']:.4f}")
print(f"   → Price increases by ${greeks['Vega']:.4f} for each 1% increase in volatility")
print(f"   → Risk exposure to volatility changes\n")

print(f"4. THETA = {greeks['Theta']:.4f}")
print(f"   → Daily time decay of ${greeks['Theta']:.4f}")
if greeks['Theta'] < 0:
    print(f"   → NEGATIVE theta: value decreases each day (unfavorable for long)")
else:
    print(f"   → POSITIVE theta: value increases each day (favorable for long)\n")

print(f"5. RHO = {greeks['Rho']:.4f}")
print(f"   → Price increases by ${greeks['Rho']:.4f} for each 1% increase in rates")
print(f"   → Risk exposure to interest rate changes")

## Section 5: Stress Testing

In [ ]:
# Volatility stress test
print("📊 STRESS TEST 1: Volatility Sensitivity")
print(f"\nWhat happens if volatility changes?\n")

mc_sim_stress = MonteCarloSimulator(S0, K, T, r, sigma, num_simulations=5000, payoff_type='call')
vol_stress = mc_sim_stress.stress_test(volatilities=np.arange(0.10, 1.0, 0.10))
print(vol_stress.to_string(index=False))

print(f"\n💡 Interpretation:")
print(f"   Higher volatility → Higher option value (positive vega)")

In [ ]:
# Interest rate stress test
print("\n📊 STRESS TEST 2: Interest Rate Sensitivity")
print(f"\nWhat happens if rates change?\n")

rate_stress = mc_sim_stress.rate_sensitivity(rates=np.arange(-0.01, 0.11, 0.02))
print(rate_stress.to_string(index=False))

print(f"\n💡 Interpretation:")
print(f"   Higher rates → Higher discount rate → Lower option value (negative rho for calls)")

## Section 6: Visualizations

In [ ]:
# Convergence plot
print("📈 Generating visualizations...")

fig = EnergyDerivativesPlotter.plot_binomial_convergence(
    S0, K, T, r, sigma,
    payoff_type='call',
    step_range=[10, 25, 50, 100, 200, 500],
    save_path='../results/01_convergence.png'
)
plt.show()
print("✓ Convergence plot saved")

In [ ]:
# MC Distribution
fig = EnergyDerivativesPlotter.plot_monte_carlo_distribution(
    S0, K, T, r, sigma,
    payoff_type='call',
    num_simulations=10000,
    save_path='../results/02_mc_distribution.png'
)
plt.show()
print("✓ Monte-Carlo distribution plot saved")

In [ ]:
# Greeks curves
fig = EnergyDerivativesPlotter.plot_greeks_curves(
    S0, K, T, r, sigma,
    payoff_type='call',
    save_path='../results/03_greeks_curves.png'
)
plt.show()
print("✓ Greeks curves plot saved")

In [ ]:
# Volatility stress test
fig = EnergyDerivativesPlotter.plot_stress_test_volatility(
    S0, K, T, r,
    payoff_type='call',
    save_path='../results/04_stress_vol.png'
)
plt.show()
print("✓ Volatility stress test plot saved")

In [ ]:
# Rate stress test
fig = EnergyDerivativesPlotter.plot_stress_test_rate(
    S0, K, T, sigma,
    payoff_type='call',
    save_path='../results/05_stress_rate.png'
)
plt.show()
print("✓ Interest rate stress test plot saved")

In [ ]:
# Method comparison
fig = EnergyDerivativesPlotter.plot_price_comparison(
    S0, K, T, r, sigma,
    payoff_type='call',
    save_path='../results/06_method_comparison.png'
)
plt.show()
print("✓ Method comparison plot saved")

## Section 7: Comprehensive Results Summary

In [ ]:
print("\n" + "="*100)
print("ENERGY DERIVATIVES PRICING FRAMEWORK - RESULTS SUMMARY".center(100))
print("="*100)

print(f"\n📊 MODEL INPUTS (from empirical CEIR data):")
print(f"  Underlying Asset (S₀):     ${S0:.4f} (energy unit price)")
print(f"  Volatility (σ):            {sigma:.2%} (annualized)")
print(f"  Time to Maturity (T):      {T} year")
print(f"  Risk-Free Rate (r):        {r:.2%}")
print(f"  Strike Price (K):          ${K:.4f} (at-the-money)")

print(f"\n💰 PRICING RESULTS:")
print(f"  Binomial Tree (N=100):           ${call_price:.4f}")
print(f"  Monte-Carlo (N=10,000):          ${mc_price:.4f} [{mc_low:.4f}, {mc_high:.4f}]")

print(f"\n📈 RISK METRICS (Greeks):")
print(f"  Delta (Δ):                 {greeks['Delta']:.4f}  (per $1 underlying)")
print(f"  Gamma (Γ):                 {greeks['Gamma']:.4f}  (per $1 underlying)")
print(f"  Vega (ν):                  {greeks['Vega']:.4f}  (per 1% volatility)")
print(f"  Theta (θ):                 {greeks['Theta']:.4f}  (daily decay)")
print(f"  Rho (ρ):                   {greeks['Rho']:.4f}  (per 1% rate)")

print(f"\n🎯 KEY FINDINGS:")
print(f"  1. Energy-backed derivatives can be priced using rigorous no-arbitrage methods")
print(f"  2. Binomial and Monte-Carlo methods converge to same value")
print(f"  3. Greeks provide complete risk profile for hedging")
print(f"  4. Stress testing shows resilience to market changes")
print(f"  5. Empirical CEIR data enables calibration to real energy economics")

print(f"\n" + "="*100)

## Section 8: Practical Applications

In [ ]:
print("\n" + "="*80)
print("PRACTICAL APPLICATIONS FOR SOLARPUNK COIN (SPK)".center(80))
print("="*80)

print(f"\n🔌 USE CASE 1: Pricing SPK Tokens")
print(f"   Fair value of 1 SPK token backed by 1 kWh of renewable energy")
print(f"   Binomial price: ${call_price:.4f}")
print(f"   Ensures no-arbitrage in token issuance and redemption\n")

print(f"🛡️ USE CASE 2: Hedging Renewable Producers")
print(f"   Producers can hedge energy price risk using SPK call options")
print(f"   Delta = {greeks['Delta']:.4f}: need {greeks['Delta']:.0%} of future production to hedge")
print(f"   Vega = {greeks['Vega']:.4f}: hedge against volatility changes\n")

print(f"💸 USE CASE 3: Stability Mechanism")
print(f"   Theta decay incentivizes holding token (earns interest over time)")
print(f"   Can implement yield farming via theta collection\n")

print(f"📊 USE CASE 4: Central Bank Integration (CBDC)")
print(f"   Rho sensitivity shows how CBDC rates affect token value")
print(f"   Formal pricing enables integration with monetary policy\n")

print(f"🌍 USE CASE 5: International Energy Markets")
print(f"   Different regions have different S₀, σ, r")
print(f"   Multi-region pricing creates arbitrage opportunities")
print(f"   Framework enables decentralized energy finance\n")

print("="*80)

## Section 9: Validation and Robustness

In [ ]:
# Validation checks
print("\n✅ MODEL VALIDATION CHECKS:\n")

# 1. Call value bounds
intrinsic = max(S0 - K, 0)
upper_bound = S0  # Can't be worth more than asset
print(f"1. OPTION VALUE BOUNDS:")
print(f"   Intrinsic value: ${intrinsic:.4f}")
print(f"   Call price: ${call_price:.4f}")
print(f"   Upper bound: ${upper_bound:.4f}")
print(f"   ✓ PASS" if intrinsic <= call_price <= upper_bound else "   ✗ FAIL")

# 2. Delta bounds
print(f"\n2. DELTA BOUNDS (should be 0 to 1):")
print(f"   Delta: {greeks['Delta']:.4f}")
print(f"   ✓ PASS" if 0 <= greeks['Delta'] <= 1 else "   ✗ FAIL")

# 3. Gamma non-negative
print(f"\n3. GAMMA NON-NEGATIVITY (should be ≥ 0):")
print(f"   Gamma: {greeks['Gamma']:.4f}")
print(f"   ✓ PASS" if greeks['Gamma'] >= 0 else "   ✗ FAIL")

# 4. Vega non-negative for calls
print(f"\n4. VEGA NON-NEGATIVITY (should be ≥ 0 for calls):")
print(f"   Vega: {greeks['Vega']:.4f}")
print(f"   ✓ PASS" if greeks['Vega'] >= 0 else "   ✗ FAIL")

# 5. Method agreement
error_pct = abs(call_price - mc_price) / call_price * 100
print(f"\n5. BINOMIAL-MONTE CARLO CONVERGENCE:")
print(f"   Binomial price: ${call_price:.4f}")
print(f"   MC price: ${mc_price:.4f}")
print(f"   Error: {error_pct:.2f}%")
print(f"   ✓ PASS" if error_pct < 1 else "   ~ Acceptable" if error_pct < 5 else "   ✗ FAIL")

## Section 10: Conclusions and Future Work

In [ ]:
print("\n" + "="*80)
print("CONCLUSIONS AND IMPLICATIONS".center(80))
print("="*80)

print(f"""
🎯 KEY CONCLUSIONS:

1. THEORETICAL RIGOR
   Energy-backed digital assets can be priced using modern finance theory
   (Black-Scholes, binomial trees, Monte-Carlo)
   No-arbitrage pricing ensures fair value

2. EMPIRICAL FOUNDATION  
   Real CEIR data from Bitcoin provides calibration
   Energy prices derived from cumulative investment ratios
   Volatility estimated from historical returns

3. PRACTICAL IMPLEMENTATION
   Binomial trees provide exact pricing for planning
   Monte-Carlo enables stress testing and portfolio analysis
   Greeks enable real-time risk management and hedging

4. POLICY RELEVANCE
   Framework bridges renewable energy economics with monetary policy
   Enables CBDC designs backed by physical assets
   Creates new energy finance instruments

5. SOLARPUNK VISION
   SPK tokens become financial instruments, not just cryptocurrencies
   Can be priced, hedged, and integrated into financial systems
   Monetizes renewable energy, improving economics

📊 FUTURE WORK:

• Multi-factor models: Include energy prices + grid utilization + storage
• Stochastic interest rates: Jump-diffusion models for rates
• Energy futures: Implied volatility from energy derivatives markets
• Portfolio optimization: Hedge ratios for energy producers
• Real implementation: Deploy on blockchain with oracles
• Regulatory framework: Coordinate with central banks and regulators

""")

print("="*80)
print(f"Analysis completed: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*80)